In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

'''
Raw Dataset을 로딩하는 과정 
+ 전체 series 데이터로부터 필요한 부분만 segmentation하는 부분 포함 
+ 1d를 2d로 reshape하는 과정 포함 
- scale up 과정 미포함
'''

'''
Path Settings
'''
CONFIGURATION_FILE_PATH = "./data/train/data_config.csv"
DATASET_PATH = "./data/train/"


'''
Figure Settings
'''
pd.set_option('display.width', 200) # for display width


'''
1. Read Configurration File (only xls)
'''
data_config = pd.read_csv(CONFIGURATION_FILE_PATH, header=0, index_col=0)
print("Configuration Dataframe dimension: ", data_config.shape)

'''
2. Read all FSR matrix data and Seat sensor data
'''
fsr_dataframe = {}
seat_dataframe = {}

for idx in data_config.index:
    fsr_filepath = DATASET_PATH+data_config.loc[idx, "fsr_matrix_1d_datafile"] # set FSR matrix data filepath
    seat_filepath = DATASET_PATH+data_config.loc[idx, "seat_datafile"] # set Seat data filepath
    print(idx, ") read data files : ", fsr_filepath, ",", seat_filepath)

    fsr_dataframe[idx] = pd.read_csv(fsr_filepath, header=0, index_col=False).iloc[:,0:162] # read FSR matrix data file
    seat_dataframe[idx] = pd.read_csv(seat_filepath, header=0, index_col=False) # read Seat data file

    # clear unnecessary columns
    del seat_dataframe[idx]['Measurement time'] # remove unnecessary column
    del fsr_dataframe[idx]['Measurement Time (sec)'] # remove unnecessary column



'''
3. Preproceess : Data Segmentation by mtime
   - @brief     FSR matrix data and Seat data should be segmented by mtime
   - @output    segmented dataframes
'''

# output dict.
fsr_dataframe_standard_segment = {}
fsr_dataframe_relax_segment = {}
seat_loadcell_dataframe_standard_segment = {}
seat_loadcell_dataframe_relax_segment = {}


for idx in data_config.index:
    mtime = data_config.loc[idx, ['standard_s_mtime', "standard_e_mtime", "relax_s_mtime", "relax_e_mtime"]]

    # seat loadcell segmentation
    seat_loadcell_dataframe_standard_segment[idx] = seat_dataframe[idx][(seat_dataframe[idx]['mtime']>=mtime.standard_s_mtime) & (seat_dataframe[idx]['mtime']<=mtime.standard_e_mtime)]
    seat_loadcell_dataframe_relax_segment[idx] = seat_dataframe[idx][(seat_dataframe[idx]['mtime']>=mtime.relax_s_mtime) & (seat_dataframe[idx]['mtime']<=mtime.relax_e_mtime)]

    # fsr matrix segmentation
    fsr_dataframe_standard_segment[idx] = fsr_dataframe[idx][(fsr_dataframe[idx]['mtime']>=mtime.standard_s_mtime) & (fsr_dataframe[idx]['mtime']<=mtime.standard_e_mtime)]
    fsr_dataframe_relax_segment[idx] = fsr_dataframe[idx][(fsr_dataframe[idx]['mtime']>=mtime.relax_s_mtime) & (fsr_dataframe[idx]['mtime']<=mtime.relax_e_mtime)]

    print("FSR Segments@Standard size : ", len(fsr_dataframe_standard_segment[idx]), ", FSR Segments@Relax size : ", len(fsr_dataframe_relax_segment[idx]))
    print("Seat Segments@Standard size : ", len(seat_loadcell_dataframe_standard_segment[idx]), ", Seat Segments@Relax size : ", len(seat_loadcell_dataframe_relax_segment[idx]))



Configuration Dataframe dimension:  (49, 12)
1 ) read data files :  ./data/train/20210303T160024-1dm.csv , ./data/train/data_01m.csv
2 ) read data files :  ./data/train/20210310T104529-1dm.csv , ./data/train/data_02m.csv
3 ) read data files :  ./data/train/20210310T110616-1dm.csv , ./data/train/data_03m.csv
4 ) read data files :  ./data/train/20210318T170103-1dm.csv , ./data/train/data_04m.csv
5 ) read data files :  ./data/train/20210310T153307-1dm.csv , ./data/train/data_05m.csv
6 ) read data files :  ./data/train/20210310T154900-1dm.csv , ./data/train/data_06m.csv
7 ) read data files :  ./data/train/20210311T131751-1dm.csv , ./data/train/data_07m.csv
8 ) read data files :  ./data/train/20210311T134516-1dm.csv , ./data/train/data_08m.csv
9 ) read data files :  ./data/train/20210311T150952-1dm.csv , ./data/train/data_09m.csv
10 ) read data files :  ./data/train/20210311T153746-1dm.csv , ./data/train/data_10m.csv
11 ) read data files :  ./data/train/20210311T154809-1dm.csv , ./data/trai

In [2]:
'''
데이터를 catrom으로 interpolation(dynamic scale)하고
좌우 데이터가 상대적으로 다른 스케일로 측정되는 문제에서
잘 나오는 쪽만 crop하여 처리하도록 함.
이때 scale up의 min max는 0~255의 절대범위가 아닌, 각 series마다 local min/max를 기준으로 한다.
+ only standard mode
'''
import os
import os.path
import gc
from skimage import io, color

DYNAMIC_SCALEUP = True


crop_standard_interpolated_paths = {}

if DYNAMIC_SCALEUP==True:
    try:
        os.mkdir("./high") # create diretory
    except FileExistsError:
        pass

    for idx in data_config.index:
        fsr_standard_segment_1d = fsr_dataframe_standard_segment[idx].iloc[:,1:161]
        fsr_standard_segment_2d = fsr_standard_segment_1d.values.reshape(-1, 16, 10) # reshape

        try:
            os.mkdir("./high/{}".format(idx)) # create diretory for each id
        except FileExistsError:
            pass


        standard_fsr_crop_file_list = []

        for ridx in range(fsr_standard_segment_2d.shape[0]):
            result_image_filepath = "./high/{}/standard_{}.jpg".format(idx, ridx)
            result_crop_image_filepath = "./high/{}/standard_crop_{}.jpg".format(idx, ridx)

            # data interpolation
            if os.path.isfile(result_image_filepath)==False:
                fig = plt.figure()
                plt.axis('off')
                plt.imshow(fsr_standard_segment_2d[ridx], interpolation='catrom', cmap='Greys_r')
                plt.savefig(result_image_filepath, bbox_inches='tight', pad_inches=0)
                plt.close()
            else:
                pass

            # crop active region
            if os.path.isfile(result_crop_image_filepath)==False:
                image = io.imread(result_image_filepath)
                grayscale = color.rgb2gray(image)
                crop = grayscale[0:grayscale.shape[0],int(grayscale.shape[1]/2):grayscale.shape[1]]
                io.imsave(result_crop_image_filepath, crop)
                standard_fsr_crop_file_list.append(result_crop_image_filepath)
                print("(standard) saved output crop images for id {}, {}".format(idx, ridx))
            else:
                pass
            
        crop_standard_interpolated_paths[idx] = pd.DataFrame(standard_fsr_crop_file_list, columns=['path'])



 1]. Convert image to uint8 prior to saving to suppress this warning.
(standard) saved output crop images for id 50, 7
(standard) saved output crop images for id 50, 8
(standard) saved output crop images for id 50, 9
(standard) saved output crop images for id 50, 10
(standard) saved output crop images for id 50, 11
(standard) saved output crop images for id 50, 12
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8

In [61]:

'''
Feature Engineering
+ 1D max pooling
'''
from skimage import io, color
import tensorflow as tf
from tensorflow.keras.layers import MaxPool1D, GlobalMaxPooling1D


crop_standard_interpolated_data = {}

max_pool_1d = tf.keras.layers.MaxPool1D(pool_size=68, strides=None, padding='valid')
max_pool_1d = tf.keras.layers.MaxPool1D(pool_size=68, strides=None, padding='valid')

image = io.imread(crop_standard_interpolated_paths[1].iloc[0]["path"])
grayscale = color.rgb2gray(image)
# print(grayscale)
# print(grayscale[0,:])
x = tf.constant(grayscale[0,:])
x = tf.reshape(x, [1, 68, 1])
a = max_pool_1d(x)
print(a)

# break
# for idx in data_config.index:
#     image = io.imread(crop_standard_interpolated_paths[1].iloc[0]["path"])
#     grayscale = color.rgb2gray(image)

#     max_pool_1d(grayscale[0,:])

    
    
    



tf.Tensor([[[227]]], shape=(1, 1, 1), dtype=uint8)
<ipython-input-61-e941719a232c>:15: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  grayscale = color.rgb2gray(image)


In [113]:
'''
draw data
'''
idx = 1
plt.close('all')
plt.figure()
plt.axis('off')
image = plt.imshow(fsr_dataframe_standard_segment[idx], interpolation='None', cmap='Greys_r')
# plt.title("Raw(Static) FSR Matrix Data", fontsize=16)
# plt.xlabel('X position(mm)')
# plt.ylabel('Y position(mm)')
# plt.tight_layout()
plt.show()

KeyError: 0